In [ ]:
using Revise

In [ ]:
import QuantumCollocation as QC
import NamedTrajectories as NT
import TrajectoryIndexingUtils as NTidx
import LinearAlgebra as LA
import SparseArrays as SA
import Plots
import Interpolations as IP
using LaTeXStrings
import QuantumOptics as QO

using FFTW:ifft, fft, fftshift, fftfreq

In [ ]:
include("utils.jl")
include("system.jl")
include("constraints.jl")
include("objectives.jl")

# using QuantumOptics package

In [ ]:
V = 10.
p_max = 10
system_momentum = ShakenLatticeSystem1D(V, p_max; bloch_basis=false)
mid = system_momentum.params[:mid]
dim = system_momentum.params[:dim]

In [ ]:
n_max = 3
ns = -n_max:n_max
v = system_momentum.params[:bloch_states][ns .+ mid,1]

In [ ]:
x_max = 200.
Nx = 5000

In [ ]:
b_position = QO.PositionBasis(-x_max, x_max, Nx)
b_momentum = QO.MomentumBasis(b_position)

In [ ]:
xs = QO.samplepoints(b_position)
ps = QO.samplepoints(b_momentum)

In [ ]:
Txp = QO.transform(b_position, b_momentum)
Tpx = QO.transform(b_momentum, b_position)

In [ ]:
p_op = QO.momentum(b_momentum)

In [ ]:
H_kin_p = p_op^2
H_kin_x = QO.LazyProduct(Txp, H_kin_p, Tpx)

In [ ]:
H_pot_I_x = QO.potentialoperator(b_position, x -> -V/2 * cos(2x))
H_pot_Q_x = QO.potentialoperator(b_position, x -> V/2 * sin(2x))
H_pot_I_p = QO.LazyProduct(Tpx, H_pot_I_x, Txp)
H_pot_Q_p = QO.LazyProduct(Tpx, H_pot_Q_x, Txp)

Z = NT.load_traj("interferometer/split_victor_opt2.jld2")

a, dts = Z.a, vec(Z.dts)
T = Z.T
times = cumsum(dts) - dts

In [ ]:
Z_split = NT.load_traj("interferometer/split_bloch78.jld2")
Z_mirror = NT.load_traj("interferometer/mirror_bloch78_Z.jld2")

In [ ]:
flight_time = 2pi * 0.5
T_flight = Int(round(flight_time/2pi * 1000; digits=0))
dts_flight = fill(flight_time/(T_flight-1), T_flight)

In [ ]:
a, dts = get_interferometer(Z_split, Z_mirror, dts_flight)
Z_time = sum(dts) - dts[1]
dts_min = minimum(dts[1:end-1])
dts_new = fill(dts_min, Int(floor(Z_time/dts_min+1, digits=0)))
a = interpolate_controls(a, dts, dts_new)
dts = dts_new
T = size(a, 2)
times = cumsum(dts) - dts

Z = NT.load_traj("end2end/traj.jld2")

a, dts = Z.a, vec(Z.dts)
flight_time = 2pi * 2.0
T_flight = Int(round(flight_time/2pi; digits=0)) * 600
jumps = [(div(Z.T, 3), flight_time), (div(2*Z.T, 3), flight_time)]
jump_Ts = fill(T_flight, 2)
a, dts = get_controls_dts(a, dts, jumps, jump_Ts)
Z_time = sum(dts) - dts[1]
dts_min = minimum(dts[1:end-1])
dts_new = fill(dts_min, Int(floor(Z_time/dts_min+1, digits=0)))
a = interpolate_controls(a, dts, dts_new)
dts = dts_new
T = size(a, 2)
times = cumsum(dts) - dts

In [ ]:
I_itp = IP.interpolate(a[1,:], IP.BSpline(IP.Cubic(IP.Free(IP.OnCell()))))
Q_itp = IP.interpolate(a[2,:], IP.BSpline(IP.Cubic(IP.Free(IP.OnCell()))))

In [ ]:
function I(t)
    if t < times[end]
        return I_itp(t/times[end]*(T-1) + 1)
    else
        return 1.0
    end
end 
function Q(t)
    if t < times[end]
        return Q_itp(t/times[end]*(T-1) + 1)
    else
        return 0.0
    end
end 

function I(t)
    return 1.0
end
function Q(t)
    return 0.0
end

In [ ]:
H_x = QO.TimeDependentSum(1.0 => H_kin_x, I => H_pot_I_x, Q => H_pot_Q_x)
H_p = QO.TimeDependentSum(1.0 => H_kin_p, I => H_pot_I_p, Q => H_pot_Q_p)

In [ ]:
function gaussian_wavepacket(x, sigma)
    return (2*sigma^2/pi)^(1/4) * exp.(-sigma^2*x.^2)
end

In [ ]:
function wave(x, n)
    return exp.(2im*x*n')
end

In [ ]:
sigma = 0.1
psi0 = gaussian_wavepacket(xs, sigma) .* (wave(xs, ns) * v)
psi0_ket = QO.Ket(b_position, psi0)

In [ ]:
Plots.plot(xs, real.(psi0_ket.data))

t_max = 20.0
dt = 0.007
times = collect(0.0:dt:t_max)

In [ ]:
times_slice = 1:length(times)

In [ ]:
Plots.plot(times[times_slice], I.(times[times_slice]))

In [ ]:
tout, psit_ket = QO.timeevolution.schroedinger_dynamic(times[times_slice], psi0_ket, H_x)

In [ ]:
psi = hcat([psi_ket.data for psi_ket in psit_ket]...)
pops = abs2.(psi)

In [ ]:
x_step = 5

In [ ]:
sum_mat = blockdiagonal(fill(ones(1, x_step)/x_step, Int(ceil(Nx/x_step)))...)[:,1:Nx]

In [ ]:
pops2 = sum_mat * pops;

In [ ]:
Plots.heatmap(tout, xs[1:x_step:end], pops2, c=Plots.cgrad([:black, :white], [0.05]))

In [ ]:
t_step = 1

In [ ]:
pops_max = maximum(pops)

In [ ]:
anim = Plots.Animation()
for t=1:t_step:length(tout)
    p = Plots.plot(xs, pops[:,t], title="$(round(tout[t], digits=2))", ylim=(0.0, pops_max))
    Plots.frame(anim, p)
end

In [ ]:
Plots.gif(anim; fps=20, loop=0)

In [ ]:
psi0_p = fourier(psi0, xs, ps; exp_sign=-1)
psi0_ket_p = QO.Ket(b_momentum, psi0_p)

In [ ]:
tout_p, psit_ket_p = QO.timeevolution.schroedinger_dynamic(times[times_slice], psi0_ket_p, H_p)

In [ ]:
psi_p = hcat([psi_ket_p.data for psi_ket_p in psit_ket_p]...)
pops_p = abs2.(psi_p)

In [ ]:
psi_p_max = maximum([maximum(abs.(real.(psi_p))), maximum(abs.(imag.(psi_p)))])
pops_p_max = maximum(pops_p)

In [ ]:
anim = Plots.Animation()
for t=1:t_step:length(tout_p)
    p = Plots.plot(ps, pops_p[:,t], title="$(round(tout_p[t], digits=1))", ylim=(0.0, pops_p_max))
    Plots.frame(anim, p)
end

In [ ]:
Plots.gif(anim; fps=20, loop=0)

In [ ]:
anim = Plots.Animation()
for t=1:t_step:length(tout_p)
    p = Plots.plot(ps, real.(psi_p[:,t]), title="$(round(tout_p[t], digits=1))", ylim=(-psi_p_max, psi_p_max))
    Plots.plot!(p, ps, imag.(psi_p[:,t]), title="$(round(tout_p[t], digits=1))", ylim=(-psi_p_max, psi_p_max))
    Plots.frame(anim, p)
end

In [ ]:
Plots.gif(anim; fps=20, loop=0)

In [ ]:
anim = Plots.Animation()
for t=1:t_step:length(tout_p)
    p = Plots.path3d(ps, real.(psi_p[:,t]), imag.(psi_p[:,t]), ylim=(-psi_p_max, psi_p_max), zlim=(-psi_p_max, psi_p_max))
    Plots.frame(anim, p)
end

In [ ]:
Plots.gif(anim; fps=20, loop=0)

# Fisher

In [ ]:
V = 10.
p_max = 10
system_momentum = ShakenLatticeSystem1D(V, p_max; bloch_basis=false)
mid = system_momentum.params[:mid]
dim = system_momentum.params[:dim]

In [ ]:
n_max = 3
ns = -n_max:n_max
v = system_momentum.params[:bloch_states][ns .+ mid,1]

In [ ]:
x_max = 400.
Nx = 4000

In [ ]:
qb_basis = QO.GenericBasis(2)

In [ ]:
qb_10 = QO.projector(QO.Ket(qb_basis, [0,1]), QO.Bra(qb_basis, [1,0]))
qb_10_lift = QO.LazyTensor(b_mom_lift, [1], (qb_10,))

In [ ]:
b_position = QO.PositionBasis(-x_max, x_max, Nx)
b_momentum = QO.MomentumBasis(b_position)

In [ ]:
b_mom_lift = QO.tensor(qb_basis, b_momentum)
b_pos_lift = QO.tensor(qb_basis, b_position)

In [ ]:
xs = QO.samplepoints(b_position)
ps = QO.samplepoints(b_momentum)

In [ ]:
Txp

In [ ]:
Txp = QO.transform(b_pos_lift, b_mom_lift)
Tpx = QO.transform(b_mom_lift, b_pos_lift)

In [ ]:
p_op = QO.LazyTensor(b_mom_lift, [2], (QO.momentum(b_momentum),))

In [ ]:
H_kin_p_lift = p_op^2

In [ ]:
H_pot_I_x = QO.potentialoperator(b_position, x -> -V/2 * cos(2x))
H_pot_Q_x = QO.potentialoperator(b_position, x -> V/2 * sin(2x))
H_pot_I_x_lift = QO.LazyTensor(b_pos_lift, [2], (H_pot_I_x,))
H_pot_Q_x_lift = QO.LazyTensor(b_pos_lift, [2], (H_pot_Q_x,))
H_pot_I_p_lift = QO.LazyProduct(Tpx, H_pot_I_x_lift, Txp)
H_pot_Q_p_lift = QO.LazyProduct(Tpx, H_pot_Q_x_lift, Txp)

In [ ]:
H_a_x = QO.potentialoperator(b_position, x -> -0.5*x)
H_a_x_lift = QO.LazyTensor(b_pos_lift, [2], (H_a_x,))
H_a_p_lift = QO.LazyProduct(Tpx, H_a_x_lift, Txp)
H_a_p_lift = QO.LazyProduct(qb_10_lift, H_a_p_lift)

Z = NT.load_traj("interferometer/split_victor_opt2.jld2")

a, dts = Z.a, vec(Z.dts)
T = Z.T
times = cumsum(dts) - dts

In [ ]:
Z_split = NT.load_traj("interferometer/split_bloch78.jld2")
Z_mirror = NT.load_traj("interferometer/mirror_bloch78_Z.jld2")

In [ ]:
flight_time = 2pi * 0.5
T_flight = Int(round(flight_time/2pi * 1000; digits=0))
dts_flight = fill(flight_time/(T_flight-1), T_flight)

In [ ]:
a, dts = get_interferometer(Z_split, Z_mirror, dts_flight)
Z_time = sum(dts) - dts[1]
dts_min = minimum(dts[1:end-1])
dts_new = fill(dts_min, Int(floor(Z_time/dts_min+1, digits=0)))
a = interpolate_controls(a, dts, dts_new)
dts = dts_new
T = size(a, 2)
times = cumsum(dts) - dts

In [ ]:
Z = NT.load_traj("end2end/traj.jld2")

In [ ]:
a, dts = Z.a, vec(Z.dts)
flight_time = 2pi * 2.0
T_flight = Int(round(flight_time/2pi; digits=0)) * 600
jumps = [(div(Z.T, 3), flight_time), (div(2*Z.T, 3), flight_time)]
jump_Ts = fill(T_flight, 2)
a, dts = get_controls_dts(a, dts, jumps, jump_Ts)
Z_time = sum(dts) - dts[1]
dts_min = minimum(dts[1:end-1])
dts_new = fill(dts_min, Int(floor(Z_time/dts_min+1, digits=0)))
a = interpolate_controls(a, dts, dts_new)
dts = dts_new
T = size(a, 2)
times = cumsum(dts) - dts

In [ ]:
I_itp = IP.interpolate(a[1,:], IP.BSpline(IP.Cubic(IP.Free(IP.OnCell()))))
Q_itp = IP.interpolate(a[2,:], IP.BSpline(IP.Cubic(IP.Free(IP.OnCell()))))

In [ ]:
function I(t)
    if t < times[end]
        return I_itp(t/times[end]*(T-1) + 1)
    else
        return 1.0
    end
end 
function Q(t)
    if t < times[end]
        return Q_itp(t/times[end]*(T-1) + 1)
    else
        return 0.0
    end
end 

function I(t)
    return 1.0
end
function Q(t)
    return 0.0
end

In [ ]:
H_p = QO.TimeDependentSum(1.0 => H_kin_p_lift, I => H_pot_I_p_lift, Q => H_pot_Q_p_lift, 1.0 => H_a_p_lift)

In [ ]:
function gaussian_wavepacket(x, sigma)
    return (2*sigma^2/pi)^(1/4) * exp.(-sigma^2*x.^2)
end

In [ ]:
function wave(x, n)
    return exp.(2im*x*n')
end

In [ ]:
sigma = 0.1
psi0 = gaussian_wavepacket(xs, sigma) .* (wave(xs, ns) * v)
psi0 = fourier(psi0, xs, ps; exp_sign=-1)
psi0_ket = QO.Ket(b_momentum, psi0)
psi0_ket = QO.tensor(QO.Ket(qb_basis, [1,0]), psi0_ket)

In [ ]:
Plots.plot(real.(psi0_ket.data[1:2:end]))

t_max = 20.0
dt = 0.007
times = collect(0.0:dt:t_max)

In [ ]:
times_slice = 1:length(times)

In [ ]:
Plots.plot(times[times_slice], I.(times[times_slice]))

In [ ]:
tout, psit_ket = QO.timeevolution.schroedinger_dynamic(times[times_slice], psi0_ket, H_p)

In [ ]:
psi = hcat([psi_ket.data for psi_ket in psit_ket]...)
psi, dpsi = psi[1:2:end,:], psi[2:2:end,:]
pops, dpops = abs2.(psi), abs2.(dpsi)

In [ ]:
Plots.plot(ps, pops[:,3001])

In [ ]:
dp = ps[2] - ps[1]

In [ ]:
function Fisher(psi, dpsi)
    eps = 0.0
    P = abs2.(psi)
    D = 2*real.(conj.(psi) .* dpsi)
    F = (1 ./ (P .+ eps))' * D.^2
    return F
end 

In [ ]:
CFI = [Fisher(psi[:,t], dpsi[:,t])*dp*pi^2 for t=1:length(tout)]

In [ ]:
Plots.plot(tout, CFI)

In [ ]:
MZFI = (8pi*(times[end]/2)^2)^2

In [ ]:
V = 10.
trunc = 11
# E_R [kHz] found in Weidner thesis
system = ShakenLatticeSystem1D(
    V, 
    trunc; 
    acc=0.0, 
    bloch_basis=true,
    bloch_transformation_trunc=2*trunc,
    include_acc_derivative=true, 
    sparse=false)
# middle index of statevector where p = 0
mid = system.params[:mid]
dim = system.params[:dim]
B = system.params[:bloch_states][:,1:trunc]
if system.params[:accelerated]
    B2 = blockdiagonal(B, B)
end

In [ ]:
function FisherB(psi, dpsi)
    psi = B*psi
    dpsi = B*dpsi
    eps = 0.0
    P = abs2.(psi)
    D = 2*real.(conj.(psi) .* dpsi)
    F = (1 ./ (P .+ eps))' * D.^2
    return F
end 

In [ ]:
Z_dts = vec(Z.dts)
Z_times = cumsum(Z_dts) - Z_dts
_, full_dts = get_controls_dts(Z.a, Z_dts, jumps, jump_Ts)
full_times = cumsum(full_dts) - full_dts

In [ ]:
psi0_iso = QC.cavity_state(0, dim)
append!(psi0_iso, zeros(length(psi0_iso)))
psi0_iso = QC.ket_to_iso(psi0_iso)

In [ ]:
psi_iso = shaken_lattice_rollout(psi0_iso, Z.a, Z_dts, system, jumps, jump_Ts)
Z_psi_dpsi = mapslices(QC.iso_to_ket, psi_iso, dims=1)

In [ ]:
Z_pops = abs2.(B*Z_psi_dpsi[1:11,:])
Z_dpops = abs2.(B*Z_psi_dpsi[12:end,:])

In [ ]:
CFI_Z = [FisherB(Z_psi_dpsi[1:11,t], Z_psi_dpsi[12:end,t]) for t=1:size(Z_psi_dpsi, 2)]

In [ ]:
p = Plots.plot(full_times, CFI_Z)
Plots.plot!(tout, CFI)

In [ ]:
B*Z_psi_dpsi[1:11,1]

In [ ]:
Nt = 849
3397/Nt, 7647/Nt

In [ ]:
anim = Plots.Animation()
t_step_fine = Int(round(size(pops, 2)/Nt, digits=0))
t_step_coarse = Int(round(size(psi_iso, 2)/Nt, digits=0))
for t=1:Nt
    p1 = Plots.plot(ps, pops[:,1+(t-1)*t_step_fine], ylim=(0.0, maximum(pops)), label=nothing)
    p2 = Plots.plot(ps, dpops[:,1+(t-1)*t_step_fine], ylim=(0.0, maximum(dpops)), label=nothing)
    p3 = Plots.plot(tout[1:1+(t-1)*t_step_fine], CFI[1:1+(t-1)*t_step_fine], xlim=(minimum(tout), maximum(tout)), ylim=(0.0, maximum(CFI)), label=nothing)

    p4 = Plots.bar(-11:11, Z_pops[:,1+(t-1)*t_step_coarse], ylim=(0.0, 1.0), label=nothing)
    p5 = Plots.bar(-11:11, Z_dpops[:,1+(t-1)*t_step_coarse], ylim=(0.0, maximum(Z_dpops)), label=nothing)
    p6 = Plots.plot(full_times[1:1+(t-1)*t_step_coarse], CFI_Z[1:1+(t-1)*t_step_coarse], xlim=(minimum(full_times), maximum(full_times)), ylim=(0.0, maximum(CFI_Z)), label=nothing)

    p = Plots.plot(p1, p2, p3, p4, p5, p6, size=(800, 1200); layout=Plots.@layout [a;b;c;d;e;f])
    Plots.frame(anim, p)
end

In [ ]:
Plots.gif(anim, "anim.gif"; fps=20, loop=0)

In [ ]:
Nt = 1000
